<style>
.red-text {
    color: red;
}
</style>

# concrete.biopython quickstart

***concrete.biopython*** is a FHE library based on python ***biopython*** library. It implements the same objects and functions when they are compatible with fhe.


### FheSeq and FheMutableSeq classes

<span style="color:orange">***FheSeq***</span>  and <span style="color:orange">***FheMutableSeq***</span>  objects are the FHE implementations of biopython <span style="color:green">***Seq***</span>  and <span style="color:green">***MutableSeq***</span>  objects.

***Biopython*** <span style="color:green">***Seq***</span> and <span style="color:green">***MutableSeq***</span>  objects are constructed from a string, generally representing a **DNA**, **RNA** or a **protein** sequence. They provide functions to process this string sequence. <span style="color:green">***MutableSeq***</span>  allows to modify this string while <span style="color:green">***Seq***</span> does not.

<span style="color:orange">***FheSeq***</span>  and <span style="color:orange">***FheMutableSeq***</span>  implement the same functions (when they are compatible with FHE) with an encrypted array of integers encoding for the string sequence.

### Working with FheSeq and FheMutableSeq objects

First of all, we need to import ***numpy*** and ***concrete.fhe***, as well as <span style="color:green">***Seq***</span>  and <span style="color:green">***MutableSeq***</span>  from ***Bio.Seq***.
Then we import <span style="color:orange">***FheSeq***</span>  and <span style="color:orange">***FheMutableSeq***</span>  from ***concrete.biopython.FheSeq***, and also ***SeqWrapper.***<span style="color:#5CC8FF">***SeqWrapper***</span> which will allow to interface <span style="color:green">***Seq***</span>  and <span style="color:green">***MutableSeq***</span>  objects with <span style="color:orange">***FheSeq***</span>  and <span style="color:orange">***FheMutableSeq***</span>  objects.

In [ ]:
import numpy as np
import numpy as np
from concrete import fhe
from Bio.Seq import Seq, MutableSeq

import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from concrete_biopython.FheSeq import FheSeq, FheMutableSeq
from concrete_biopython.SeqWrapper import SeqWrapper

Let's define an arbitrary function ***process_seq*** that takes in input a <span style="color:green">***Seq***</span>  and a <span style="color:green">***MutableSeq***</span>  objects ***seq1*** and ***seq2***, and processes them using some of the possibilities offered by the biopython library:

In [ ]:
def process_seq(seq1, seq2):
    seq2.pop()
    new_seq = seq1.reverse_complement() + seq2[0:3]
    protein = new_seq.translate()
    print(new_seq)
    return protein

Let's create two sequences and test this function:

In [ ]:
seq1 = Seq('ACCAGGTAC')
seq2 = MutableSeq('CGTTAGC')
output_seq = process_seq(seq1, seq2)
print(output_seq)

### Processing steps with FHE

Now, we will see how we can run the function ***process_seq*** homomorphically, using <span style="color:orange">***FheSeq***</span>  and a <span style="color:orange">***FheMutableSeq***</span>  objects. 

We will need to follow the steps below:
1. convert <span style="color:green">***Seq***</span>  and a <span style="color:green">***MutableSeq***</span>  objects to integer arrays with <span style="color:#5CC8FF">***SeqWrapper***</span>***.toIntegers***
2. encrypt the integer arrays
3. create <span style="color:orange">***FheSeq***</span>  and a <span style="color:orange">***FheMutableSeq***</span>  objects from the encrypted integer arrays inside the circuit
4. call our function ***process_seq*** homomorphically on the FHE sequence objects
5. Convert back the output from a <span style="color:orange">***FheSeq***</span>  to an encrypted array with <span style="color:orange">***FheSeq***</span>***.toArray***
6. Decrypt the encrypted output array
7. convert the array back to a <span style="color:green">***Seq***</span> object using <span style="color:#5CC8FF">***SeqWrapper***</span>***.toSeq***

<div>
<img src="https://rcd-media.com/docs/fhe/diagram-im-seq.png" width="650"/>
</div>


### Factorization
For steps **1**, **2**, **6** and **7**, we can create a circuit wrapper to do the Seq to integer and integer to Seq conversions in a FHE-compatible way :

In [ ]:
# wrap a circuit ( see the 'circuit' as an arbitrary function ) in order to input and output Bio.Seq objects
def circuit_wrapper(circuit, seq1, seq2, simulate=False):
    # convert Seq objects to integers with SeqWrapper.toIntegers
    integers1 = SeqWrapper(seq1).toIntegers()
    integers2 = SeqWrapper(seq2).toIntegers()
    
    # run the circuit with integer inputs
    integer_output = circuit.simulate(integers1, integers2) if simulate else circuit.encrypt_run_decrypt(integers1, integers2)

    # convert back the integer outputs into a Seq objects with SeqWrapper.toSeq
    return SeqWrapper(integer_output).toSeq()

For steps **3** and **5**, we can create an adapter function that will be the circuit's main function. This function will create <span style="color:orange">***FheSeq***</span>  and <span style="color:orange">***FheMutableSeq***</span>  objects for the function ***process_seq*** to process homomorphically. Then it will convert back the <span style="color:orange">***FheSeq***</span>  output into an encrypted integer array that the circuit can decrypt.

In [ ]:
def process_seq_adapter(integer_seq1, integer_seq2):
    
    # convert integer sequences into FheSeq and FheMutableSeq objects
    seq1=FheSeq(integer_seq1)
    seq2=FheMutableSeq(integer_seq2)
    
    # process the sequence objects with our function
    new_seq = process_seq(seq1, seq2)
    
    # return the new sequence as integer array
    return new_seq.toArray()

For step **4**, let's make the circuit from ***process_seq_adapter*** function, and create a correct inputset taking into account the number of possible letters with ***SeqWrapper.maxInteger()*** and the size of input sequences with ***len(seq1)*** and ***len(seq2)***:

In [ ]:
# compile the process_seq_adapter function and create a circuit
compiler = fhe.Compiler(lambda data1,data2: process_seq_adapter(data1, data2), {"data1": "encrypted", "data2": "encrypted"})
circuit = compiler.compile(
    inputset=[
    (np.random.randint(0, SeqWrapper.maxInteger()+1, size=(len(seq1),)),
    np.random.randint(0, SeqWrapper.maxInteger()+1, size=(len(seq2),)))
    for _ in range(100)
    ],
    configuration=fhe.Configuration(
        enable_unsafe_features=True,
        use_insecure_key_cache=True,
        insecure_key_cache_location=".keys",
        dataflow_parallelize=False, # setting it to True makes the jupyter kernel crash
    ),
    verbose=False,
)

### Execution 

We can now run our wrapped circuit on the variables ***seq1*** and ***seq2*** defined earlier and compare the output sequence with the one obtained earlier:

In [ ]:
# now we can run our circuit on Seq objects and compare the result with output_seq

# with simulation
fheSim_output_seq = circuit_wrapper(circuit, seq1, seq2, True)
print('Simulated :', fheSim_output_seq)
assert(output_seq == fheSim_output_seq)

# and without (slower)
fhe_output_seq = circuit_wrapper(circuit, seq1, seq2, False)
print('FHE :', fhe_output_seq)
assert(output_seq == fhe_output_seq)